In [1]:
import tkinter as tk
from tkinter import Button, Canvas, PhotoImage, Label, Entry, OptionMenu
import numpy as np
import pandas as pd
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import math

In [2]:
global start
global end

In [3]:
def load_data(file_name, start_point=0, end_point=60):
    global t, cement, furnace_slag, fly_ash, water, plasticizer, coarse_aggerate, fine_aggerate, age, output
    
    new_df = pd.read_csv(file_name)
    new_np = new_df.to_numpy()
    
    t = np.arange(new_np[start_point:end_point, 0].size)
    cement = new_np[start_point:end_point, 0].reshape(-1, 1)
    furnace_slag = new_np[start_point:end_point, 1].reshape(-1, 1)
    fly_ash = new_np[start_point:end_point, 2].reshape(-1, 1)
    water = new_np[start_point:end_point, 3].reshape(-1, 1)
    plasticizer = new_np[start_point:end_point, 4].reshape(-1, 1)
    coarse_aggerate = new_np[start_point:end_point, 5].reshape(-1, 1)
    fine_aggerate = new_np[start_point:end_point, 6].reshape(-1, 1)
    age = new_np[start_point:end_point, 7].reshape(-1, 1)
    output = new_np[start_point:end_point, 8].reshape(-1, 1)

In [4]:
def plot_regression(yhat, MSE, model_name):
    fig = Figure(figsize=(5, 4), dpi=100)
    plot = fig.add_subplot(1, 1, 1)
    plot.scatter(t + start, output, color="black", label="_nolegend_")
    plot.plot(t + start, output, color="black")
    plot.plot(t + start, yhat, "--", color="#FF004D")
    plot.legend(["Strength", f"{model_name} Modelling Strength"])
    
    canvas_widget = FigureCanvasTkAgg(fig, master=window)
    canvas_widget.get_tk_widget().grid(column=1, row=0)
    
    linear_mse = Label(text=f"{model_name} MSE = {float(MSE)}\n", background="white", width=50, height=4)
    linear_mse.grid(column=1, row=1)
    
    window.update_idletasks()

In [5]:
def plot_every_model():
    yhat1, MSE1 = Linear_modeling()
    yhat2, MSE2 = Polynomial_modeling()
    yhat3, MSE3 = Mixed_modelling()
    
    fig = Figure(figsize=(5, 4), dpi=100)
    plot = fig.add_subplot(1, 1, 1)
    plot.scatter(t + start, output, color="black", label="_nolegend_")
    plot.plot(t + start, output, color="black")
    plot.plot(t + start, yhat1, "--", color="#FF004D")
    plot.plot(t + start, yhat2, "--", color="#570A57")
    plot.plot(t + start, yhat3, "--", color="#FC5404")
    plot.legend(["Strength", "Linear Model", "Polynomial Model", "Mixed Model"])
    
    lines_img.__del__()
    
    all_mse = tk.Label(text=f"Linear MSE = {float(MSE1)}\nPolynomial MSE = {float(MSE2)}\nMixed MSE = {float(MSE3)}", 
                       background="white", width=50, height=4)
    all_mse.grid(column=1, row=1)

    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas_widget = canvas.get_tk_widget()
    canvas_widget.grid(column=1, row=0)

    window.update_idletasks()

In [6]:
def Linear_modeling():
    Fi1 = np.zeros((output.size, 10))
    
    for i in range(output.size):
        Fi1[i, :] = [1, t[i], cement[i][0], furnace_slag[i][0], fly_ash[i][0], 
                     water[i][0], plasticizer[i][0],
                     coarse_aggerate[i][0], fine_aggerate[i][0], age[i][0]]   
    
    A1 = Fi1.T @ Fi1
    B1 = Fi1.T @ output
    teta1 = np.linalg.pinv(A1) @ B1
    yhat1 = Fi1 @ teta1
    e1 = output - yhat1
    MSE1 = e1.T @ e1 / (2 * output.size)
    
    return yhat1, MSE1

In [7]:
def Polynomial_modeling():
    Fi2 = np.zeros((output.size, 19))

    for i in range(output.size):
        Fi2[i, :] = [1, t[i], t[i]*t[i], 
                     cement[i][0], cement[i][0]*cement[i][0], furnace_slag[i][0],furnace_slag[i][0]*furnace_slag[i][0], 
                     fly_ash[i][0], fly_ash[i][0]*fly_ash[i][0], water[i][0],water[i][0]*water[i][0], 
                     plasticizer[i][0], plasticizer[i][0]*plasticizer[i][0], coarse_aggerate[i][0],coarse_aggerate[i][0]*coarse_aggerate[i][0], 
                     fine_aggerate[i][0], fine_aggerate[i][0]*fine_aggerate[i][0], age[i][0], age[i][0]*age[i][0]]

    
    A2 = Fi2.T @ Fi2
    B2 = Fi2.T @ output
    teta2 = np.linalg.pinv(A2) @ B2
    yhat2 = Fi2 @ teta2
    e2 = output - yhat2
    MSE2 = e2.T @ e2 / (2 * output.size)
    
    return yhat2, MSE2

In [8]:
def Mixed_modelling():
    Fi3 = np.zeros((output.size, 37))
    
    for i in range(output.size):
        Fi3[i, :] = [1, t[i], math.cos(t[i]), math.exp(-t[i]), math.sqrt(abs(t[i])), 
                      cement[i][0], math.cos(cement[i][0]), math.exp(-cement[i][0]), math.sqrt(abs(cement[i][0])),
                      furnace_slag[i][0], math.cos(furnace_slag[i][0]), math.exp(-furnace_slag[i][0]), math.sqrt(abs(furnace_slag[i][0])),
                      fly_ash[i][0], math.cos(fly_ash[i][0]), math.exp(-fly_ash[i][0]), math.sqrt(abs(fly_ash[i][0])),
                      water[i][0], math.cos(water[i][0]), math.exp(-water[i][0]), math.sqrt(abs(water[i][0])),
                      plasticizer[i][0], math.cos(plasticizer[i][0]), math.exp(-plasticizer[i][0]), math.sqrt(abs(plasticizer[i][0])),
                      coarse_aggerate[i][0], math.cos(coarse_aggerate[i][0]), math.exp(-coarse_aggerate[i][0]), math.sqrt(abs(coarse_aggerate[i][0])),
                      fine_aggerate[i][0], math.cos(fine_aggerate[i][0]), math.exp(-fine_aggerate[i][0]), math.sqrt(abs(fine_aggerate[i][0])),
                      age[i][0], math.cos(age[i][0]), math.exp(-age[i][0]), math.sqrt(abs(age[i][0])),]
    
    A3 = Fi3.T @ Fi3
    B3 = Fi3.T @ output
    teta3 = np.linalg.pinv(A3) @ B3
    yhat3 = Fi3 @ teta3
    e3 = output - yhat3
    MSE3 = e3.T @ e3 / (2 * output.size)
    
    return yhat3, MSE3

In [9]:
def load_button():
    global start, end
    
    beginning = int(start_entry.get())
    ending = int(end_entry.get())
    
    start = beginning
    end = ending
    
    load_data("concrete_data.csv", beginning, ending)

In [10]:
start = 0
end = 30

load_data("concrete_data.csv", start, end)

In [11]:
window = tk.Tk()
window.config(padx=30, pady=60, bg="white")
window.title("Linear Regression")

canvas = Canvas(window, width=300, height=300, bg="white", highlightthickness=0)
lines_img = PhotoImage(file="lines.png")
canvas.create_image(150, 150, image=lines_img)
canvas.grid(column=1, row=0)

set_data = Button(window, text="Set Data", command=load_button, width=8)
set_data.grid(column=1, row=3, sticky="e")

start_label = Label(window, text="Start Point", background="white")
start_label.grid(column=1, row=2, sticky="w")

start_entry = Entry(window, width=10)
start_entry.focus()
start_entry.insert(0, "0")
start_entry.grid(column=1, row=3, sticky="w")

end_label = Label(window, text="End Point", background="white")
end_label.grid(column=1, row=2)

end_entry = Entry(window, width=10)
end_entry.insert(0, "30")
end_entry.grid(column=1, row=3)

Linear_button = Button(window, text="Linear \nModelling\nRegression", command=lambda: plot_regression(*Linear_modeling(), model_name="Linear"))
Linear_button.grid(column=1, row=5, sticky="w", pady=30)

Polynomial_button = Button(window, text="Polynomial \nModelling\nRegression", command=lambda: plot_regression(*Polynomial_modeling(), model_name="Polynomial"))
Polynomial_button.grid(column=1, row=5, pady=30)

Mixed_button = Button(window, text="Mixed \nModelling\nRegression", command=lambda: plot_regression(*Mixed_modelling(), model_name="Mixed"))
Mixed_button.grid(column=1, row=5, sticky="e", pady=30)

plot_every_button = Button(window, text="Plot Every Model Together", command=plot_every_model)
plot_every_button.grid(column=1, row=6)

window.mainloop()

C:\Users\goktu\AppData\Local\Temp\ipykernel_10036\2233360907.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  linear_mse = Label(text=f"{model_name} MSE = {float(MSE)}\n", background="white", width=50, height=4)
C:\Users\goktu\AppData\Local\Temp\ipykernel_10036\3830337708.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_mse = tk.Label(text=f"Linear MSE = {float(MSE1)}\nPolynomial MSE = {float(MSE2)}\nMixed MSE = {float(MSE3)}",
